In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Label

# --- Function: Estimate rectangular tiling ---
def estimate_rectangular_tiling(x_dim_mm, y_dim_mm, pixel_size_um, sensor_pixels, overlap_percent):
    pixel_size_mm = pixel_size_um / 1000  # µm to mm
    tile_length_mm = pixel_size_mm * sensor_pixels
    step_size = tile_length_mm * (1 - overlap_percent / 100)

    tiles_x = int(np.ceil((x_dim_mm - tile_length_mm) / step_size + 1))
    tiles_y = int(np.ceil((y_dim_mm - tile_length_mm) / step_size + 1))

    total_tiles = tiles_x * tiles_y
    return total_tiles, tiles_x, tiles_y

# --- Function: Compute dimensions from ratios ---
def compute_dimensions(volume_mm3, rx, ry, rz):
    total_ratio = rx * ry * rz
    scale = (volume_mm3 / total_ratio) ** (1/3)
    x_dim = scale * rx
    y_dim = scale * ry
    z_dim = scale * rz
    return x_dim, y_dim, z_dim

# --- Function: Calculate data size ---
def calculate_data_size(
    desired_volume,
    overlap_percent,
    expansion_factor,
    XY_voxel_size_nm,
    Z_step,
    sensor_pixels,
    bit_depth,
    num_channels,
    rx,
    ry,
    rz
):
    pixel_size_um = XY_voxel_size_nm / 1000  # nm to µm
    z_step_mm = Z_step / 1e6

    expanded_volume_mm3 = desired_volume * (expansion_factor ** 3)
    x_dim, y_dim, z_dim = compute_dimensions(expanded_volume_mm3, rx, ry, rz)

    total_tiles, tiles_x, tiles_y = estimate_rectangular_tiling(
        x_dim, y_dim, pixel_size_um, sensor_pixels, overlap_percent
    )
    total_slices = int(np.ceil(z_dim / z_step_mm))

    pixels_per_plane = sensor_pixels ** 2
    total_planes = total_tiles * total_slices * num_channels
    total_voxel = pixels_per_plane * total_planes

    bytes_per_voxel = bit_depth / 8
    total_bytes = total_voxel * bytes_per_voxel

    return total_bytes, total_tiles, total_slices, x_dim, y_dim, z_dim

# --- Display Widget ---
dimension_display = widgets.HTML()

# --- Update and plot ---
def update_plot_and_info(
    current_volume,
    overlap_percent,
    expansion_factor,
    XY_voxel_size_nm,
    Z_step,
    sensor_pixels,
    bit_depth,
    num_channels,
    rx,
    ry,
    rz
):
    volume_range = np.logspace(np.log10(0.1), np.log10(500), 300)
    raw_bytes = []

    for v in volume_range:
        b, *_ = calculate_data_size(
            v, overlap_percent, expansion_factor, XY_voxel_size_nm,
            Z_step, sensor_pixels, bit_depth, num_channels, rx, ry, rz
        )
        raw_bytes.append(b)

    use_pb = max(raw_bytes) > 1e15
    unit = "PB" if use_pb else "TB"
    scale_factor = 1e15 if use_pb else 1e12
    data_sizes = [b / scale_factor for b in raw_bytes]

    selected_bytes, total_tiles, total_slices, x_dim, y_dim, z_dim = calculate_data_size(
        current_volume, overlap_percent, expansion_factor, XY_voxel_size_nm,
        Z_step, sensor_pixels, bit_depth, num_channels, rx, ry, rz
    )
    selected_data = selected_bytes / scale_factor

    actual_dims = compute_dimensions(current_volume, rx, ry, rz)
    expanded_dims = compute_dimensions(current_volume * (expansion_factor ** 3), rx, ry, rz)

    dimension_display.value = f"""
    <b>Actual Dimensions (mm):</b> X={actual_dims[0]:.2f}, Y={actual_dims[1]:.2f}, Z={actual_dims[2]:.2f}<br>
    <b>Expanded Dimensions (mm):</b> X={expanded_dims[0]:.2f}, Y={expanded_dims[1]:.2f}, Z={expanded_dims[2]:.2f}<br>
    <b>Nmber of Tiles per acquisition :</b> {total_tiles:,} &nbsp;&nbsp;&nbsp; <b>Total Slices:</b> {total_slices:,}<br>
    <b>Estimated Size:</b> {selected_data:.2f} {unit}
    """

    plt.figure(figsize=(10, 6))
    plt.plot(volume_range, data_sizes, label='Estimated Data Size')
    plt.axvline(current_volume, linestyle='--', color='red',
                label=f'Selected Volume = {current_volume:.2f} mm³\nSize = {selected_data:.2f} {unit}')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel("Volume (mm³, log scale)")
    plt.ylabel(f"Data Size ({unit}, log scale)")
    plt.title("Data Size vs Volume")
    plt.grid(True, which="both", linestyle='--', linewidth=0.5)
    plt.legend()
    plt.show()

# --- Widgets ---
current_volume = widgets.FloatLogSlider(value=1, base=10, min=np.log10(0.1), max=np.log10(500), step=0.01, continuous_update=False)
volume_label = widgets.HTML(value="<b>Target volume (mm³):</b>")
volume_widget = widgets.VBox([volume_label, current_volume])
overlap_percent = widgets.BoundedFloatText(value=10, min=0, max=100, step=1, description='Overlap (%)')
expansion_factor = widgets.BoundedIntText(value=16, min=1, max=100, step=1, description='Expansion Factor')
XY_voxel_size_nm = widgets.BoundedIntText(value=150, min=1, max=10000, step=10, description='Pixel size (nm)')
Z_step = widgets.BoundedIntText(value=400, min=1, max=10000, step=10, description='Z-step (nm)')
sensor_pixels = widgets.BoundedIntText(value=2046, min=1, max=10000, step=1, description='Sensor Pixels')
bit_depth = widgets.BoundedIntText(value=12, min=1, max=32, step=1, description='Bit Depth')
num_channels = widgets.BoundedIntText(value=2, min=1, max=100, step=1, description='Channels')
ratio_label =  Label("Desired dimension ratio (X:Y:Z):")
parameters_label =  Label("Imaging parameters:")
rx = widgets.BoundedFloatText(value=1, min=0.1, max=10, step=0.1, description='Ratio x')
ry = widgets.BoundedFloatText(value=1, min=0.1, max=10, step=0.1, description='Ratio y')
rz = widgets.BoundedFloatText(value=1, min=0.1, max=10, step=0.1, description='Ratio z')

controls = {
    'current_volume': current_volume,
    'overlap_percent': overlap_percent,
    'expansion_factor': expansion_factor,
    'XY_voxel_size_nm': XY_voxel_size_nm,
    'Z_step': Z_step,
    'sensor_pixels': sensor_pixels,
    'bit_depth': bit_depth,
    'num_channels': num_channels,
    'rx': rx,
    'ry': ry,
    'rz': rz
}
ui = widgets.VBox([
    widgets.VBox([
        volume_widget,
        parameters_label,
        overlap_percent,
        expansion_factor,
        XY_voxel_size_nm,
        Z_step,
        sensor_pixels,
        bit_depth,
        num_channels,
        ratio_label,
    ]),
    widgets.HBox([rx, ry, rz]),
    dimension_display
])


out = widgets.interactive_output(update_plot_and_info, controls)
display(ui, out)


Output()